In [ ]:
# Until v4.26.0 is out, requires:
# pip install --upgrade git+https://github.com/huggingface/transformers
# Sometimes, I may have to use bleeding edge:
# pip install -U git+https://github.com/huggingface/diffusers

import time

import sdcommon
import depth2img

def run(imagename="out/txt2img00000.png",
        prompt="There are flowers everywhere",
        neg_prompt = "Disfigured. Too many fingers.",
        **kwargs):
    ml = depth2img.ML()
    params = depth2img.Params(imagename=imagename, prompt=prompt, neg_prompt=neg_prompt, **kwargs)
    start = time.time()
    img, depth = ml.run(params) #, local_files_only=False)
    print("Took %.1fs" % (time.time()-start))
    # todo: convert to pil, depth))
    print("Saved as %s.png" % sdcommon.save(ml, params, img[0]))
    return img[0]

img = run(prompt=prompt, steps=50, strength=0.9)
img